In [1]:
import numpy as np
from keras.datasets import cifar10
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm_notebook as tqdm
from keras.layers import Layer, Input, Conv2D, MaxPool2D, Flatten, Dense, Reshape
from keras.models import Model
from src.gaussian_attention_layer import VisualAttentionLayer

C:\Users\Roland\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load data

In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
oh_enc = OneHotEncoder(n_values=10, sparse=False)
y_train = oh_enc.fit_transform(y_train)
y_test = oh_enc.transform(y_test)

x_train = x_train/128.0-1.0
x_test = x_test/128.0-1.0

### Create Model

In [10]:
inputs = Input(shape=(32,32,3))

x = Conv2D(64, kernel_size=(3,3), activation="relu")(inputs)
x = MaxPool2D()(x)
x = Conv2D(64, kernel_size=(3,3), activation="relu")(x)
x = MaxPool2D()(x)
x = Flatten()(x)
x = Dense(6, activation="relu")(x)
x = VisualAttentionLayer(output_dim=(20,20))([inputs, x])

x = Conv2D(64, kernel_size=(3,3), activation="relu")(x)
x = MaxPool2D()(x)
x = Conv2D(64, kernel_size=(3,3), activation="relu")(x)
x = Flatten()(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 30, 30, 64)   1792        input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_11 (MaxPooling2D) (None, 15, 15, 64)   0           conv2d_14[0][0]                  
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 13, 13, 64)   36928       max_pooling2d_11[0][0]           
__________________________________________________________________________________________________
max_poolin

### Train the Model

In [12]:
history = model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=2, validation_data=(x_test, y_test))  

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
 - 15s - loss: 1.2769 - acc: 0.5512 - val_loss: 1.3119 - val_acc: 0.5489
Epoch 2/5
 - 14s - loss: 1.1688 - acc: 0.5913 - val_loss: 1.2212 - val_acc: 0.5865
Epoch 3/5
 - 14s - loss: 1.1024 - acc: 0.6181 - val_loss: 1.1458 - val_acc: 0.6033
Epoch 4/5
 - 14s - loss: 1.0520 - acc: 0.6366 - val_loss: 1.3841 - val_acc: 0.5381
Epoch 5/5
 - 15s - loss: 1.0168 - acc: 0.6481 - val_loss: 1.1687 - val_acc: 0.6009
